## stacking 集成算法：生成次级学习器数据集
基础学习器抽取特征，次级学习器融合。5折。

In [5]:
# 导入必要的工具包
import numpy as np #用于数值计算
import pandas as pd #用于数据表处理，数据文件读写

In [4]:
path_train_all = '../data/new/train=all/'
path_train = '../data/new/train=predict/'
train_name = '%strain=pred-%s-%s.csv'
path_test = '../data/new/test=predict/'
test_name = '%stest=pred-%s-%s.csv'

val_path = '../data/val/' #保存校验集测试结果
val_name = '%sval=stacking-base-%s-%s.csv'
summit_path = '../data/summit/' #保存提交文件，测试集测试结果 
out_name = '%ssummit=stacking-base-%s-%s.csv' # 生成的结果文件名称

train_part_num = 5 #训练集拆分个数
target = 'prediction_pay_price'
user_id = 'user_id'

## 1.读取校验集预测结果，0,1,2,3,4合并。

In [18]:
# # model_name = 'RidgeCV'
# model_name = 'RandomForestRegressor'
# for i in range(train_part_num):
#     if i == 0:
#         col_df = pd.read_csv(val_name % (val_path, model_name, i))
#     else:
#         col_df = pd.concat([col_df, pd.read_csv(val_name % (val_path, model_name, i))])
#     print('col_df.shape=', col_df.shape)
# col_df.sort_values(user_id, inplace=True)
# col_df.head()

In [23]:
# val=stacking-base-RidgeCV-0
model_list = ['RidgeCV', 'RandomForestRegressor','lasso','GradientBoostingRegressor','ExtraTreesRegressor',
              'ElasticNetCV','DNN_V2', 'BaggingRegressor']
for j, model_name in enumerate(model_list): #每个模型预测一个特征 
#     print('j=%s model_name=%s' % (j, model_name))
    for i in range(train_part_num): # 读取校验集预测结果，0,1,2,3,4合并。
        if i == 0:
            col_df = pd.read_csv(val_name % (val_path, model_name, i))
        else:
            col_df = pd.concat([col_df, pd.read_csv(val_name % (val_path, model_name, i))])
#         print('col_df.shape=', col_df.shape)
    col_df.sort_values(user_id, inplace=True) #必须排序保持一致 
    if j == 0:
        train_df = pd.DataFrame(col_df[user_id], columns=[user_id], dtype=np.int32)
    train_df['pred_%s'%j] = col_df[target].astype(np.float32) #多个特征合并，得到整个数据集。
    print('train_df.shape=', train_df.shape)

train_df.shape= (2288006, 2)
train_df.shape= (2288006, 3)
train_df.shape= (2288006, 4)
train_df.shape= (2288006, 5)
train_df.shape= (2288006, 6)
train_df.shape= (2288006, 7)
train_df.shape= (2288006, 8)
train_df.shape= (2288006, 9)


In [24]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2288006 entries, 218050 to 86498
Data columns (total 9 columns):
user_id    int32
pred_0     float32
pred_1     float32
pred_2     float32
pred_3     float32
pred_4     float32
pred_5     float32
pred_6     float32
pred_7     float32
dtypes: float32(8), int32(1)
memory usage: 96.0 MB


In [25]:
# 预测特征 v1 特征个数 =9
train_df.to_csv(train_name %(path_train, '1', 'col=9'), index=False)
train_df.head()

,user_id,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7
218050,1,0.535776,0.000076,0.784902,0.509253,0.000087,0.714253,0.000000,0.000044
99015,1593,0.000000,0.027313,0.076759,0.509253,0.043894,0.015648,0.000000,0.066684
282730,1594,0.000000,0.000310,0.000000,0.459161,0.023473,0.000000,0.045323,0.000217
298802,1596,0.173117,0.000138,0.000000,0.548669,0.000069,0.000000,0.719231,0.000227
197794,1598,0.000000,0.001624,0.000000,0.459161,0.025087,0.000000,0.124845,0.001590


## 2.读取测试集预测结果，0,1,2,3,4平均。

In [46]:
# model_name = 'RandomForestRegressor'
# for i in range(train_part_num):
#     if i == 0:
#         col_df1 = pd.read_csv(out_name % (summit_path, model_name, i))
#         col_df1.sort_values(user_id, inplace=True) #必须排序保持一致 
#     else:
#         col_df2 = pd.read_csv(out_name % (summit_path, model_name, i))
#         col_df2.sort_values(user_id, inplace=True) #必须排序保持一致 
# #         print('i=', i)
# #         print(col_df1.head())
# #         print(col_df2.head())
#         col_df1[target] = (col_df1[target] + col_df2[target]) / 2
# #         print(col_df1.head())
# col_df1.head() #两种方式结果不同 

In [48]:
# model_name = 'RandomForestRegressor'
# for i in range(train_part_num):
#     if i == 0:
#         col_df1 = pd.read_csv(out_name % (summit_path, model_name, i))
#     else:
#         col_df2 = pd.read_csv(out_name % (summit_path, model_name, i))
# #         print('i=', i)
# #         print(col_df1.head())
# #         print(col_df2.head())
#         col_df1[target] = (col_df1[target] + col_df2[target])
# #         print(col_df1.head())
# col_df1[target] = (col_df1[target] / train_part_num)
# col_df1.head() #这种方式更好 

In [49]:
model_list = ['RidgeCV', 'RandomForestRegressor','lasso','GradientBoostingRegressor','ExtraTreesRegressor',
              'ElasticNetCV','DNN_V2', 'BaggingRegressor']
for j, model_name in enumerate(model_list): #每个模型预测一个特征 
#     print('j=%s model_name=%s' % (j, model_name))
    for i in range(train_part_num):
        if i == 0:
            col_df1 = pd.read_csv(out_name % (summit_path, model_name, i))
        else:
            col_df2 = pd.read_csv(out_name % (summit_path, model_name, i))
#             print('i=', i)
#             print(col_df1.head())
#             print(col_df2.head())
            col_df1[target] = (col_df1[target] + col_df2[target])
#             print(col_df1.head())
    col_df1[target] = (col_df1[target] / train_part_num) #求平均
    if j == 0:
        test_df = pd.DataFrame(col_df1[user_id], columns=[user_id], dtype=np.int32)
    test_df['pred_%s'%j] = col_df1[target].astype(np.float32) #多个特征合并，得到整个数据集。
    print('test_df.shape=', test_df.shape)

test_df.shape= (828934, 2)
test_df.shape= (828934, 3)
test_df.shape= (828934, 4)
test_df.shape= (828934, 5)
test_df.shape= (828934, 6)
test_df.shape= (828934, 7)
test_df.shape= (828934, 8)
test_df.shape= (828934, 9)


In [50]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 828934 entries, 0 to 828933
Data columns (total 9 columns):
user_id    828934 non-null int32
pred_0     828934 non-null float32
pred_1     828934 non-null float32
pred_2     828934 non-null float32
pred_3     828934 non-null float32
pred_4     828934 non-null float32
pred_5     828934 non-null float32
pred_6     828934 non-null float32
pred_7     828934 non-null float32
dtypes: float32(8), int32(1)
memory usage: 28.5 MB


In [51]:
# 预测特征 v1 特征个数 =9
test_df.to_csv(test_name %(path_test, '1', 'col=9'), index=False)
test_df.head()

,user_id,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7
0,14933,0.071611,0.000618,0.013686,0.468971,0.167516,0.000000,0.701374,0.000203
1,14934,0.247105,0.000097,0.021984,0.468971,0.000023,0.009713,1.939122,0.000105
2,14935,0.545124,0.000098,0.000000,0.468971,0.006006,0.000000,1.332410,0.000098
3,14936,0.965462,0.016567,0.000000,0.468971,0.003039,0.000000,3.096730,0.000076
4,14937,0.035901,0.000117,0.000000,0.468971,0.006005,0.000000,1.349272,0.000101


## 3.给训练集添加 target 

In [8]:
train_pred = pd.read_csv(train_name %(path_train, '1', 'col=9'))
train_pred.shape

(2288006, 9)

In [12]:
train_pred.sort_values(user_id, inplace=True) #必须排序保持一致 
train_pred.head()

,user_id,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7
0,1,0.535776,0.000076,0.784902,0.509253,0.000087,0.714253,0.000000,0.000044
1,1593,0.000000,0.027313,0.076759,0.509253,0.043894,0.015648,0.000000,0.066684
2,1594,0.000000,0.000310,0.000000,0.459161,0.023473,0.000000,0.045323,0.000217
3,1596,0.173117,0.000138,0.000000,0.548669,0.000069,0.000000,0.719231,0.000227
4,1598,0.000000,0.001624,0.000000,0.459161,0.025087,0.000000,0.124845,0.001590


In [6]:
train_list = []
for i in range(train_part_num):
    cur_id = i + 1
    train_list.append(pd.read_csv('%strain=fe-%s.csv' % (path_train_all, cur_id)))
train_fe = pd.concat(train_list)
train_fe.shape

(2288006, 109)

In [15]:
train_fe2 = train_fe[[user_id, 'prediction_pay_price']]
train_fe2.sort_values(user_id, inplace=True) #必须排序保持一致 
train_fe2.head()

,user_id,prediction_pay_price
218050,1,0.0
99015,1593,0.0
282730,1594,0.0
298802,1596,0.0
197794,1598,0.0


In [16]:
# 看看有没有不同 id 
print(train_fe2[user_id].value_counts())

2047       2
1509025    2
1834814    2
1830716    2
1810226    2
1781540    2
1771297    2
1722121    2
1724168    2
1400554    2
1392358    2
1386213    2
1382115    2
1380064    2
1371868    2
1349331    2
1337037    2
1314496    2
1556150    2
1547954    2
1521319    2
1576768    2
1591113    2
1595211    2
1904608    2
2232320    2
1964031    2
1959933    2
1949686    2
1933294    2
          ..
3078638    1
3076591    1
3017200    1
3015153    1
757635     1
3023349    1
3029494    1
3027447    1
778121     1
3097049    1
3099096    1
3092951    1
786317     1
2892213    1
2898358    1
2896311    1
784268     1
2900409    1
2906554    1
2904507    1
2908605    1
3144142    1
3142095    1
3082704    1
3086802    1
3084755    1
3090900    1
774027     1
771978     1
6147       1
Name: user_id, Length: 1830405, dtype: int64
